In [2]:
import cv2
import os
import numpy as np
import pickle
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

Let us first read the train and test files

In [4]:
train_images_filenames = pickle.load(open('train_images_filenames.dat','rb'))
test_images_filenames = pickle.load(open('test_images_filenames.dat','rb'))
train_images_filenames = ['..' + n[15:] for n in train_images_filenames]
test_images_filenames  = ['..' + n[15:] for n in test_images_filenames]
train_labels = pickle.load(open('train_labels.dat','rb')) 
test_labels = pickle.load(open('test_labels.dat','rb'))

n_train = len(train_images_filenames)
n_test = len(test_images_filenames)
tradeoff = n_train/(n_train+n_test)
print(f"There are {tradeoff*100}% of training images.)

We create a KAZE/SIFT/.... object detector and descriptor

In [5]:
Detector = cv2.SIFT_create(nfeatures=2000)
#Detector = cv2.KAZE_create(threshold=0.0001)

In [31]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)

We compute the  descriptors for all the train images and subsequently build a numpy array with all the descriptors stacked together

In [7]:
for file in range(len(train_images_filenames)):
    train_images_filenames[file] = train_images_filenames[file][3:]

for file in range(len(test_images_filenames)):
    test_images_filenames[file] = test_images_filenames[file][3:]

In [25]:
Train_descriptors = []
Train_label_per_descriptor = []

for filename,labels in zip(train_images_filenames,train_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=Detector.detectAndCompute(gray,None)
    Train_descriptors.append(des)
    Train_label_per_descriptor.append(labels)


(930117, 128)


In [30]:
Test_descriptors = []
Test_label_per_descriptor = []

for filename,labels in zip(test_images_filenames,test_labels):
    ima=cv2.imread(filename)
    gray=cv2.cvtColor(ima,cv2.COLOR_BGR2GRAY)
    kpt,des=Detector.detectAndCompute(gray,None)
    Test_descriptors.append(des)
    Test_label_per_descriptor.append(labels)


In [ ]:
with open("pkls/test_des_sift", "wb") as file:
    pickle.dump(test_des, file)
with open("pkls/train_des_sift", "wb") as file:
    pickle.dump(train_des, file)

Compare the matches of each test image to training images and set the label with most matches.

In [43]:
test_results = []

for test_label, test_des in zip(Test_label_per_descriptor, Test_descriptors):
    best_num_match = 0
    best_match_label = None
    for train_label, train_des in zip(Train_label_per_descriptor, Train_descriptors):
        total_matches = 0
        matches = bf.knnMatch(test_des, train_des, k=2)

        # Apply ratio test
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
        total_matches == len(good_matches)

        if total_matches > best_num_match:
            best_num_match = total_matches
            best_match_label = train_label

    test_results.append(best_match_label)

Save the resutls as pkl.

In [ ]:
with open("pkls/test_matches_sift", "wb") as file:
    pickle.dump(test_results, file)


Check Accuracy

In [ ]:
num_corrects = 0
for test_label in Test_label_per_descriptor:
    num_corrects += 1 if test_label == test_results else 0

acc = num_corrects/len(Test_label_per_descriptor)

F1-Score

In [ ]:
classes = list(set(Test_label_per_descriptor))

for classe in classes:
    for test_label, result_label in zip(Test_label_per_descriptor, test_results):
        TP += 1 if test_label == classe and result_label == classe else 0
        FP += 1 if test_label != classe and result_label == classe else 0
        FN += 1 if test_label == classe and result_label != classe else 0

    P_i = TP / (TP + FP)
    R_i = TP / (TP + FN)
    f1_i += 2 * (P_i * R_i) / (P_i + R_i)  
    
f1 = f1_i / len(classes)
print(f"F1 score = {f1}")
            
